In [1]:
import numpy as np
import pandas as pd
from astropy.table import Table, vstack
from matplotlib import pyplot as plt
from glob import glob
from astropy.stats import LombScargle
from scipy import stats
import warnings

%matplotlib inline

In [2]:
data = 'massive_2min.csv'
data_dir = '../data/massive_lcs/'
massive = pd.read_csv(data)

In [3]:
def get_lc_from_id(ticid, normalize=True, clean=True):
    
    """
    Step 1: load in the light curve from the TIC number
    
    Parameters
    ----------
    
    ticid : int
        TIC number
    
    normalize : bool, default True
        If True, median divides each Sector's data, creates a column 'NormPDCSAP_FLUX'
        
    clean : bool, default True
        If True, selects only data with QUALITY = 0
        
    Returns
    -------
    
    outlc : `pandas.DataFrame`
        Pandas DataFrame containing the lightcurve, with all sector's lightcurves appended
    
    """
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        files = glob(data_dir+'*{}*'.format(ticid))
        if len(files) == 0:
            raise ValueError('TIC number not recognized')
        elif len(files) > 2:
            raise ValueError('TIC number ambiguous')
        else:
            lcs = []
            for f in files:
                lc = Table.read(f, format='fits')
                if clean:
                    lc = lc[lc['QUALITY'] == 0]
                if normalize:
                    lc['NormPDCSAP_FLUX'] = lc['PDCSAP_FLUX']/np.nanmedian(lc['PDCSAP_FLUX'])
                lcs.append(lc)
            outlc = vstack(lcs)
            return outlc.to_pandas()

In [4]:
def peak_finder(f,p,n,FAL=0,width=20):
    """Given frequency/power periodogram, find the first n peaks with power greater than FAL"""
    
    assert (len(f) == len(p))&(len(f) > width),"Length of frequency array and power array much be equal, and >20"
    
    peaks = []
    strengths = []
    
    for i in range(width,len(f) - width):
        
        if np.all(p[i] > p[i -width:i])&np.all(p[i] > p[i + 1:i+width+1])&(p[i]>FAL):
            
            peaks.append(f[i])
            strengths.append(p[i])
    
    isort = np.argsort(strengths)
    peaks = np.array(peaks)[isort]
    strengths = np.array(strengths)[isort]
    return peaks[::-1][:n],strengths[::-1][:n]

In [5]:
def extract_periodic_components(ticid, phaseplot = True, dynamicplot = True, pmin = 0.1, pmax = 28.0, **kwargs):
    
    """
    Step 2: Identify and separate periodic features.
    
    Parameters
    ----------
    ticid : int
        TIC number
        
    phaseplot : bool, default True
        If True, for each significant period above white noise, subtracts the other periods, then phase folds the data and plot
        
    dynamicplot : bool, default True
        If True, for each significant period above white noise, subtracts the other periods, plots phase vs. cycle number
        
    pmin : float, default 0.1
        Minimum period to search for in the data
    
    pmax : float, default 28.0
        Maximum period to search for in the data
        
    **kwargs
        Passed to get_lc_from_id
        
    Returns
    -------
    
    Ps : array-like
        Array of significant periods above white noise 
        
    ts : array-like
        Times of lightcurve
        
    flux : array-like
        Flux
    
    whitened_flux : array-like
        Flux after getting rid of the periodic components
    
    """
    
    #Book keeping
    name = np.unique(massive['CommonName'][massive['ticid']==ticid])[0]
    spt = np.unique(massive['SpT'][massive['ticid']==ticid])[0]
    

    #Assemble the lightcurve, plus the smoothed light curve, get rid of NaNs
    lc = get_lc_from_id(ticid, **kwargs)
    slc = lc.rolling(128, center=True).median()
    
    rtime = lc['TIME'].values
    if 'NormPDCSAP_FLUX' in lc.columns:
        rflux = lc['NormPDCSAP_FLUX'].values
    else:
        rflux = lc['PDCSAP_FLUX'].values
    
    time = rtime[~np.isnan(rflux)]
    flux = rflux[~np.isnan(rflux)]
    
    srtime = slc['TIME'].values
    if 'NormPDCSAP_FLUX' in lc.columns:
        srflux = slc['NormPDCSAP_FLUX'].values
    else:
        srflux = slc['PDCSAP_FLUX'].values
    
    stime = srtime[~np.isnan(srflux)]
    sflux = srflux[~np.isnan(srflux)]
    
    LS_obj = LombScargle(time, flux)
    
    f, p = LS_obj.autopower(minimum_frequency=1.0/pmax,
                            maximum_frequency=1.0/pmin)
    
    #Assume the null hypothesis is white noise, find peaks with FAP < 0.001
    FAL = LS_obj.false_alarm_level(0.01)
    
    fs, ps = peak_finder(f, p, 10, FAL=FAL, width=10)
    
    plt.plot(1.0/f,p)
    plt.scatter(1.0/fs,ps)
    plt.xlabel('Period [d]')
    plt.ylabel('Power')
    plt.title('{0}: {1}'.format(name,spt))
    plt.savefig('{0}_periodogram.png'.format(name.replace('*','').replace(' ','')))
    plt.close('all')
    
    #Copy the flux array to return later after we subtract off all the signals
    whitened_flux = flux.copy()
    for i,freq in enumerate(fs):
        
        #make a copy of the flux - more significant peaks
        temp_flux = whitened_flux.copy()
        
        #subtract off the model for this frequency from the temp flux array and also the 
        #whitened flux
        sub_ls = LombScargle(time,temp_flux)
        temp_model = sub_ls.model(time, freq)
        temp_flux -= temp_model
        whitened_flux -= temp_model

        #iterate over less significant peaks
        other_freqs = fs[i+1:]

        for of in other_freqs:
            #subtract off the model for these frequencies
            temp_ls = LombScargle(time, temp_flux)
            mod = temp_ls.model(time, of)
            temp_flux -= mod
        
        #add the model back in
        temp_flux += temp_model
        
        #Only plot the biggest 3 components
        if i < 3:
            phase = (time*freq) % 1
            if phaseplot:
                lc_df = pd.DataFrame(data={'Phase':phase,'Flux':temp_flux}).sort_values('Phase')
                lc_df_smooth = lc_df.rolling(128, center=True).median()
                plt.scatter(lc_df['Phase'],lc_df['Flux'],s=1)
                plt.plot(lc_df_smooth['Phase'],lc_df_smooth['Flux'],c='C1')
                plt.xlabel('Phase')
                plt.ylabel('Flux - Other Periods')
                plt.xlim(0,1)
                plt.ylim(0.9999*np.min(temp_flux),1.0001*np.max(temp_flux))
                plt.title('{0}: {1}. P={2}'.format(name,spt,1.0/freq))
                plt.savefig('{0}_phase_P{1}.png'.format(name.replace('*','').replace(' ',''),i))
                plt.close('all')
                
            if dynamicplot:
                cycle = np.floor((time-np.min(time))*freq)
                ybins = len(np.unique(cycle))
                xbins = 100
                H, xedges, yedges, binnumber = stats.binned_statistic_2d(phase, cycle, temp_flux, statistic='median', bins=(xbins,ybins))  
                H = np.ma.masked_where(H==0, H) #masking where there was no data
                XX, YY = np.meshgrid(xedges, (ybins/(ybins-1))*yedges+0.5)
                colobj = plt.pcolormesh(XX,YY,H.T)
                plt.xlabel('Phase')
                plt.ylabel('Cycle')
                plt.colorbar(colobj, ax=plt.gca(), label='Median Flux Per Bin')
                plt.title('{0}: {1}. P={2}'.format(name,spt,1.0/freq))
                plt.savefig('{0}_dynamic_P{1}.png'.format(name.replace('*','').replace(' ',''),i))
                plt.close('all')
                
    return 1.0/fs, time, flux, whitened_flux

In [6]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for tic in np.unique(massive['ticid']):
        print(tic)
        extract_periodic_components(tic)

29281992
29984014
31530164
55295028
55401985
126658353
150392584
160192386
167304735
167720039
179305185
220424200
232083010
255686390
269794986
278611926
279957111
281741629
307160124
370227780
389076668
389437365
410447919
